In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from aiidalab.home import AiidalabHome
import urllib.parse as urlparse

In [ ]:
home = AiidalabHome()

In [ ]:
%%javascript
// in case an app was moved up or down
// I need to remove the corresponding variable from the url
// othervise if I update page again the same move will happen

var url = window.location.toString();
function getQueryVariable(variable)
{
       var query = window.location.search.substring(1);
       var vars = query.split("&");
       for (var i=0;i<vars.length;i++) {
               var pair = vars[i].split("=");
               if(pair[0] == variable){return pair[1];}
       }
       return(false);
}
if (name = getQueryVariable("move_up")){
    url = url.replace("move_up=".concat(name), "");
} else if (name = getQueryVariable("move_down")) {
    url = url.replace("move_down=".concat(name), "");
}
window.history.pushState("", "", url);

In [ ]:
parsed_url = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)
if 'move_up' in parsed_url:
    home.move_updown(parsed_url['move_up'][0], -1)
elif 'move_down' in parsed_url:
    home.move_updown(parsed_url['move_down'][0], +1)

In [ ]:
display(home.render())